In [1]:
import cobra
import libsbml
from cobra.core import Metabolite, Reaction
import pandas as pd
from cobra import flux_analysis
ModelF = cobra.io.read_sbml_model("CAM_12PModel.xml")
ModelF.solver="glpk"
cobra.flux_analysis.pfba(ModelF)

#adding maintenance cost
PPFD = 100                            #light intensity of the model
ATPase = (0.0049*PPFD) + 2.7851       #non-growth assocaited maintenance (NGAM) cost based on light - see Topfer et al 2020 Supplemental information section 1.2.3

for i in range(1,13):
    ModelF.reactions.get_by_id("ATPase_tx"+str(i)).lower_bound = ATPase
    ModelF.reactions.get_by_id("ATPase_tx"+str(i)).upper_bound = ATPase
    
    #Setting NADPH demand to 1/3 of ATP demand and distributing this demand to cytosol, plastid and mitochondria based on Cheung et al 2013 (doi: 10.1111/tpj.12252)
    ModelF.reactions.get_by_id("NADPHoxc_tx"+str(i)).lower_bound = ATPase/9
    ModelF.reactions.get_by_id("NADPHoxc_tx"+str(i)).upper_bound = ATPase/9
    
    ModelF.reactions.get_by_id("NADPHoxp_tx"+str(i)).lower_bound = ATPase/9
    ModelF.reactions.get_by_id("NADPHoxp_tx"+str(i)).upper_bound = ATPase/9
    
    ModelF.reactions.get_by_id("NADPHoxm_tx"+str(i)).lower_bound = ATPase/9
    ModelF.reactions.get_by_id("NADPHoxm_tx"+str(i)).upper_bound = ATPase/9
        
cobra.flux_analysis.pfba(ModelF)

4.450782702947559
-------
0.42999264936692144
-------
Phloem_output_tx1	-0.3097224281318271
0.0022970297 4_AMINO_BUTYRATE_c1 + 0.0004186704 ARG_c1 + 0.0015049505 ASN_c1 + 0.0004186704 CYS_c1 + 0.0792079208 FRU_c1 + 0.0693069307 GLC_c1 + 0.0240792079 GLN_c1 + 0.0124356436 GLT_c1 + 0.0007128713 GLY_c1 + 0.0004186704 HIS_c1 + 0.0017425743 ILE_c1 + 0.0020594059 LEU_c1 + 0.0022178218 LYS_c1 + 0.0038811881 L_ALPHA_ALANINE_c1 + 0.006019802 L_ASPARTATE_c1 + 0.0004186704 MET_c1 + 0.0057029703 PHE_c1 + 0.9603960396 PROTON_e1 + 0.0004186704 PRO_c1 + 0.003960396 SER_c1 + 0.0068910891 THR_c1 + 0.0004186704 TRP_c1 + 0.0004186704 TYR_c1 + 0.0027722772 VAL_c1 + 0.7326732673 sSUCROSE_b1 --> 0.9603960396 PROTON_c1 + Phloem_e1
-------
PROTON_ATPase_c1	0.3105325415071109
0.65 ATP_c1 + 0.45 PROTON_c1 + WATER_c1 + 0.35 aATP_c1 --> 0.5 ADP_c1 + PROTON_e1 + 0.7 Pi_c1 + 0.5 aADP_c1 + 0.3 aPi_c1
-------
5.8315578352997575
-------
0.35750555178863164


In [2]:
ModelF3 = ModelF.copy() 

In [3]:
#Apply constraints
#Constrain ATP_ADP_Pi_pc to 0 (we know NTT, plastidic nucleotide transporter is only active in importing ATP to chloroplast at night or in non-photosynthetic tissues (Reinhold et al., 2007; Flugge et al., 2011; Voon and Lim, 2019))

for i in range(1,7):    
    ModelF3.reactions.get_by_id("ATP_ADP_Pi_pc"+str(i)).lower_bound = 0
    ModelF3.reactions.get_by_id("ATP_ADP_Pi_pc"+str(i)).upper_bound = 0
    
#Constrain PEPCK to 0 (because we are modelling Phalaenopsis) (ME takes over as decarboxylating enzyme)    
    
for i in range(1,13):    
    ModelF3.reactions.get_by_id("PEPCARBOXYKIN_RXN_c"+str(i)).lower_bound = 0
    ModelF3.reactions.get_by_id("PEPCARBOXYKIN_RXN_c"+str(i)).upper_bound = 0

#Remove reaction H_pc (because this was included as a hypothetical reaction)

for i in range(1,13):    
    ModelF3.reactions.get_by_id("H_pc"+str(i)).lower_bound = 0
    ModelF3.reactions.get_by_id("H_pc"+str(i)).upper_bound = 0

#Change direction of PYR-H symporter allowing only PYR import into mitonchondria (Le et al., 2022)

for i in range(1,13):    
    ModelF3.reactions.get_by_id("PYRUVATE_PROTON_mc"+str(i)).lower_bound = -1000
    ModelF3.reactions.get_by_id("PYRUVATE_PROTON_mc"+str(i)).upper_bound = 0
    
#Add PYR channel reaction to re-allow PYR export from mitochondria
for i in range(1,13):
    rxn = Reaction("PYR_mc"+str(i)+"_channel",name = "PYR_mc"+str(i)+"_channel")
   
    rxn.add_metabolites({ModelF3.metabolites.get_by_id("PYRUVATE_m"+str(i)):-1,
                         ModelF3.metabolites.get_by_id("PYRUVATE_c"+str(i)):1})
   
    rxn.lower_bound = 0
    rxn.upper_bound = 1000
    ModelF3.add_reaction(rxn)

#Constrain a flux ratio between NADP-ME and NAD-ME according to enzyme activity measurements in Kalanchoë and Dever et al. 2015
for i in range(1,13):
    new_constraint1 = ModelF3.problem.Constraint(
        8*(ModelF3.reactions.get_by_id("MALIC_NADP_RXN_c"+str(i)).flux_expression + ModelF3.reactions.get_by_id("MALIC_NADP_RXN_p"+str(i)).flux_expression) - ModelF3.reactions.get_by_id("1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m"+str(i)).flux_expression,
        lb=0,
        ub=0)
    ModelF3.add_cons_vars(new_constraint1)
    
#Constrain a flux ratio between PPDK_c and PPDK_p according to Kondo et al. 2000 and Dever et al. 2015
for i in range(1,13):
    new_constraint2 = ModelF3.problem.Constraint(
        2*ModelF3.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p"+str(i)).flux_expression - ModelF3.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_c"+str(i)).flux_expression,
        lb=0,
        ub=0)
    ModelF3.add_cons_vars(new_constraint2)

In [8]:
sol3 = flux_analysis.parsimonious.pfba(ModelF3)
cobra.io.write_sbml_model(ModelF3, "CAM_12P_ME_Model.xml")

In [23]:
#summarizing top 5 proton producing and consuming reactions during daytime and nighttime relative to the cytosol
#@Sanu --> make a function of this so that we can add the function after each part where we change and check the impact of a specific constraint? 

#Daytime
fout = open("DaytimeProducing.csv","w")
for i in range(1,7):
    met = ModelF3.metabolites.get_by_id("PROTON_c"+str(i))
    for rxn in met.reactions:
        if round(rxn.flux,3)!=0 and rxn.flux*rxn.metabolites[met]>0:
            fout.write(rxn.id+","+str(((rxn.flux*rxn.metabolites[met])*60*60*2)/1000)+"\n") #convert per sec to per 2h (60*60*2) --> mmol/m-2/2h
                        
fout.close()

df = pd.read_csv('DaytimeProducing.csv', header=None)
df.rename(columns={0: 'RxnID', 1: 'DaytimeFlux'}, inplace=True)
df['RxnID'] = df['RxnID'].map(lambda x: str(x)[:-1])
df.to_csv('DaytimeProducing.csv', index=False)
dff = df.groupby(['RxnID']).DaytimeFlux.sum().reset_index()
dff.sort_values("DaytimeFlux", axis=0, ascending=False, inplace=True, na_position='first') 
dff.to_csv('DaytimeProducing.csv', index=True)


fout = open("DaytimeConsuming.csv","w")
for i in range(1,7):
    met = ModelF3.metabolites.get_by_id("PROTON_c"+str(i))
    for rxn in met.reactions:
        if round(rxn.flux,3)!=0 and rxn.flux*rxn.metabolites[met]<0:
            fout.write(rxn.id+","+str(((rxn.flux*rxn.metabolites[met])*60*60*2)/1000)+"\n") 
                        
fout.close()

df = pd.read_csv('DaytimeConsuming.csv', header=None)
df.rename(columns={0: 'RxnID', 1: 'DaytimeFlux'}, inplace=True)
df['RxnID'] = df['RxnID'].map(lambda x: str(x)[:-1])
df.to_csv('DaytimeConsuming.csv', index=False)
dff = df.groupby(['RxnID']).DaytimeFlux.sum().reset_index()
dff.sort_values("DaytimeFlux", axis=0, ascending=True, inplace=True, na_position='first') 
dff.to_csv('DaytimeConsuming.csv', index=True)

#Nighttime
fout = open("NighttimeProducing.csv","w")
for i in range(7,13):
    met = ModelF3.metabolites.get_by_id("PROTON_c"+str(i))
    for rxn in met.reactions:
        if round(rxn.flux,3)!=0 and rxn.flux*rxn.metabolites[met]>0:
            fout.write(rxn.id+","+str(((rxn.flux*rxn.metabolites[met])*60*60*2)/1000)+"\n")
                        
fout.close()

df = pd.read_csv('NighttimeProducing.csv', header=None)
df.rename(columns={0: 'RxnID', 1: 'NighttimeFlux'}, inplace=True)
df['RxnID'] = df['RxnID'].map(lambda x: str(x)[:-1])
df['RxnID'] = df['RxnID'].map(lambda x: x.rstrip('1'))
df.to_csv('NighttimeProducing.csv', index=False)
dff = df.groupby(['RxnID']).NighttimeFlux.sum().reset_index()
dff.sort_values("NighttimeFlux", axis=0, ascending=False, inplace=True, na_position='first') 
dff.to_csv('NighttimeProducing.csv', index=True)

fout = open("NighttimeConsuming.csv","w")
for i in range(7,13):
    met = ModelF3.metabolites.get_by_id("PROTON_c"+str(i))
    for rxn in met.reactions:
        if round(rxn.flux,3)!=0 and rxn.flux*rxn.metabolites[met]<0:
            fout.write(rxn.id+","+str(((rxn.flux*rxn.metabolites[met])*60*60*2)/1000)+"\n")
                        
fout.close()

df = pd.read_csv('NighttimeConsuming.csv', header=None)
df.rename(columns={0: 'RxnID', 1: 'NighttimeFlux'}, inplace=True)
df['RxnID'] = df['RxnID'].map(lambda x: str(x)[:-1])
df['RxnID'] = df['RxnID'].map(lambda x: x.rstrip('1'))
df.to_csv('NighttimeConsuming.csv', index=False)
dff = df.groupby(['RxnID']).NighttimeFlux.sum().reset_index()
dff.sort_values("NighttimeFlux", axis=0, ascending=True, inplace=True, na_position='first') 
dff.to_csv('NighttimeConsuming.csv', index=True)

In [43]:
#write csv file with all reactions of all model phases included
fout = open("pFBA_output_PiCactive.csv","w")

for rxn in ModelF3.reactions:
    fout.write(rxn.id+","+rxn.reaction+","+rxn.name+","+str(rxn.flux)+"\n")
    
fout.close()

In [44]:
#@Sanu: In the manuscript, FVA is included in the sensitivity analysis part --> so also move this part to the notebook with sensitivy analyses? (see bottom of that notebook) and do it on modelF3?
def customFVA(ModelF3,rxnlist = ["Photon_tx",]):
    opt = ModelF3.slim_optimize()
    if ModelF3.solver.status!='optimal':
        print("FBA solution is not optimal")
        return    
    
    from cobra.flux_analysis import pfba
    SoF = pfba(ModelF3)
    backup = ModelF3.copy()
    
    objectives = list()
    for rxn in ModelF3.reactions:
        if rxn.objective_coefficient!=0:
            objectives.append(rxn.id)
    
    
    fva_dict = dict()
    if len(objectives)!=1:
        print("Cannot run this simple function for multiobjective models")
        return
    else:
        rxn = ModelF3.reactions.get_by_id(objectives[0])
        rxn.objective_coefficient = 0
        rxn.upper_bound = opt
        rxn.lower_bound = opt
        backup2 = ModelF3.copy()
        tempModel = backup2.copy()

        from cobra.flux_analysis.parsimonious import add_pfba
        prob = tempModel.problem
        with tempModel:
            add_pfba(tempModel, fraction_of_optimum=0)
            ub = tempModel.slim_optimize(error_value=None)
            flux_sum = prob.Variable("flux_sum", ub=1 * ub)
            flux_sum_constraint = prob.Constraint(
                tempModel.solver.objective.expression - flux_sum,
                lb=0,
                ub=0,
                name="flux_sum_constraint",
            )
        tempModel.add_cons_vars([flux_sum, flux_sum_constraint])
        
        for rxnID in rxnlist:
            tempModel.reactions.get_by_id(rxnID).objective_coefficient = 1
            maxim = round(tempModel.slim_optimize(),6)
            tempModel.reactions.get_by_id(rxnID).objective_coefficient = -1
            minim = round(-1*tempModel.slim_optimize(),6)
            tempModel.reactions.get_by_id(rxnID).objective_coefficient = 0
            fva_dict[rxnID]=(maxim, minim)
    return fva_dict

fva_dict = customFVA(ModelF3)

KeyError: 'Photon_tx'

In [ ]:
fva_sol=customFVA(ModelF3,rxnlist = ["MAL_PROTON_rev_vc1","MAL_PROTON_rev_vc2", "MAL_PROTON_rev_vc3", "MAL_PROTON_rev_vc4", "MAL_PROTON_rev_vc5", "MAL_PROTON_rev_vc6", "Pi_PROTON_mc1","Pi_PROTON_mc2","Pi_PROTON_mc3","Pi_PROTON_mc4","Pi_PROTON_mc5", "Pi_PROTON_mc6", "GAPOXNPHOSPHN_RXN_c1", "GAPOXNPHOSPHN_RXN_c2","GAPOXNPHOSPHN_RXN_c3","GAPOXNPHOSPHN_RXN_c4","GAPOXNPHOSPHN_RXN_c5","GAPOXNPHOSPHN_RXN_c6"])

In [ ]:
fva_sol=pd.DataFrame(fva_sol)
output_file="FVA.xlsx"
fva_sol.to_excel(output_file, index=False)

In [ ]:
#Additional checks that are reported in the manuscript

ModelF4 = ModelF3.copy()

# Check importance of PiC transporter 
for i in range(1,13):
    ModelF4.reactions.get_by_id("Pi_PROTON_mc"+str(i)).lower_bound = 0
    ModelF4.reactions.get_by_id("Pi_PROTON_mc"+str(i)).upper_bound = 0
    
#Also constrain reaction H_mc to carry zero flux in this case (because this was included as a hypothetical reaction)
for i in range(1,13):    
    ModelF4.reactions.get_by_id("H_mc"+str(i)).lower_bound = 0
    ModelF4.reactions.get_by_id("H_mc"+str(i)).upper_bound = 0

sol4 = flux_analysis.parsimonious.pfba(ModelF4)

#write csv file with all reactions of all model phases included
fout = open("pFBA_output_PiCoff.csv","w")

for rxn in ModelF4.reactions:
    fout.write(rxn.id+","+rxn.reaction+","+rxn.name+","+str(rxn.flux)+"\n")
    
fout.close()

In [ ]:
#Simulate different mitochondrial pyruvate export mechanisms (symport and antiport)
#Uncomment one to simulate, leave the other commented

#@Sanu: do we need to put the channel mechanism back on 0 as well, as the channel is still active in ModelF4? 
    
#PYR-H symport reaction
# for i in range(1,13):
#     rxn = Reaction("PYR_H_mc"+str(i)+"_symport",name = "PYR_H_mc"+str(i)+"_symport")
   
#     rxn.add_metabolites({ModelF4.metabolites.get_by_id("PYRUVATE_m"+str(i)):-1,
#                          ModelF4.metabolites.get_by_id("PROTON_m"+str(i)):-1,
#                          ModelF4.metabolites.get_by_id("PYRUVATE_c"+str(i)):1,
#                          ModelF4.metabolites.get_by_id("PROTON_c"+str(i)):1})
   
#     rxn.lower_bound = 0
#     rxn.upper_bound = 1000
#     ModelF4.add_reaction(rxn)

# PYR-H antiport reaction
# for i in range(1,13):
#     rxn = Reaction("PYR_H_mc"+str(i)+"_antiport",name = "PYR_H_mc"+str(i)+"_antiport")
   
#     rxn.add_metabolites({ModelF4.metabolites.get_by_id("PYRUVATE_m"+str(i)):-1,
#                          ModelF4.metabolites.get_by_id("PROTON_m"+str(i)):1,
#                          ModelF4.metabolites.get_by_id("PYRUVATE_c"+str(i)):1,
#                          ModelF4.metabolites.get_by_id("PROTON_c"+str(i)):-1})
   
#     rxn.lower_bound = 0
#     rxn.upper_bound = 1000
#     ModelF4.add_reaction(rxn)

#Also constrain reaction H_mc to carry zero flux in this case (because this was included as a hypothetical reaction)
for i in range(1,13):    
    ModelF4.reactions.get_by_id("H_mc"+str(i)).lower_bound = 0
    ModelF4.reactions.get_by_id("H_mc"+str(i)).upper_bound = 0
    
sol4 = flux_analysis.parsimonious.pfba(ModelF4)

In [1]:
#Summary of fluxes through particular CAM reactions when checking different mitochondrial pyruvate export mechanisms
starch = 0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("GLYCOGENSYN_RXN_p"+str(i))
    #print(rxn.reaction)
    #print(rxn.flux)
    starch = starch+rxn.flux
print("---------------")
print("Total starch synthesized: "+str(starch))

MPC=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("PYRUVATE_PROTON_mc"+str(i))
    #print(rxn.reaction)
    #print(rxn.flux)
    MPC=MPC+rxn.flux
print("---------------")
print("Total flux through MPC: "+str(MPC))

#uncomment one of the mechanisms to check flux
# PYRsymport=0
# for i in range(1,13):
#     rxn = ModelF4.reactions.get_by_id("PYR_H_mc"+str(i)+"_symport")
# #     print(rxn.reaction)
# #     print(rxn.flux)
#     PYRsymport=PYRsymport+rxn.flux
# print("---------------")
# print("Total flux through PYRsymport: "+str(PYRsymport))

# PYRantiport=0
# for i in range(1,13):
#     rxn = ModelF4.reactions.get_by_id("PYR_H_mc"+str(i)+"_antiport")
#     #print(rxn.reaction)
#     #print(rxn.flux)
#     PYRantiport=PYRantiport+rxn.flux
# print("---------------")
# print("Total flux through PYRantiport: "+str(PYRantiport))

# PYRchannel=0
# for i in range(1,13):
#     rxn = ModelF4.reactions.get_by_id("PYR_mc"+str(i)+"_channel")
# #     print(rxn.reaction)
# #     print(rxn.flux)
#     PYRchannel=PYRchannel+rxn.flux
# print("---------------")
# print("Total flux through PYRchannel: "+str(PYRchannel))

PEPC=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("PEPCARBOX_RXN_c"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    PEPC=PEPC+rxn.flux
print("---------------")
print("Total flux through PEPC: "+str(PEPC))

NADME=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("1_PERIOD_1_PERIOD_1_PERIOD_39_RXN_m"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    NADME=NADME+rxn.flux
print("---------------")
print("Total flux through NADME: "+str(NADME))

NADPMEcyt=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("MALIC_NADP_RXN_c"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    NADPMEcyt=NADPMEcyt+rxn.flux
print("---------------")
print("Total flux through NADPMEcyt: "+str(NADPMEcyt))

NADPMEplast=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("MALIC_NADP_RXN_p"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    NADPMEplast=NADPMEplast+rxn.flux
print("---------------")
print("Total flux through NADPMEplast: "+str(NADPMEplast))

PPDKcyt=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_c"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    PPDKcyt=PPDKcyt+rxn.flux
print("---------------")
print("Total flux through PPDKcyt: "+str(PPDKcyt))

PPDKplast=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("PYRUVATEORTHOPHOSPHATE_DIKINASE_RXN_p"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    PPDKplast=PPDKplast+rxn.flux
print("---------------")
print("Total flux through PPDKplast: "+str(PPDKplast))

RubOxy=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("RXN_961_p"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    RubOxy=RubOxy+rxn.flux
print("---------------")
print("Total flux through RubOxy: "+str(RubOxy))

RubCarboxy=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("RIBULOSE_BISPHOSPHATE_CARBOXYLASE_RXN_p"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    RubCarboxy=RubCarboxy+rxn.flux
print("---------------")
print("Total flux through RubCarboxy: "+str(RubCarboxy))

MalateEfflux=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("MAL_PROTON_rev_vc"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    MalateEfflux=MalateEfflux+rxn.flux
print("---------------")
print("Total flux through MalateEfflux: "+str(MalateEfflux))

PiC=0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("Pi_PROTON_mc"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    PiC=PiC+rxn.flux
print("---------------")
print("Total flux through PiC: "+str(PiC))

MalImp=0
for i in range (1,13):  
    rxn = ModelF4.reactions.get_by_id("MAL_PROTON_vc"+str(i))
#     print(rxn.reaction)
#     print(rxn.flux)
    MalImp=MalImp+rxn.flux
print("---------------")
print("Total malate imported in vacuole: "+str(MalImp))

ATPdayMito = 0
for i in range(1,13):
    rxn = ModelF4.reactions.get_by_id("Mitochondrial_ATP_Synthase_m"+str(i))
    ATPdayMito += rxn.flux
print("---------------")
print("Total flux through ATP synthase in mitochondria:" +str(ATPdayMito))

phloem_rxn = ModelF4.reactions.get_by_id("Diel_phloem_export")
print("---------------")
print("Diel phloem export flux: " +str(phloem_rxn.flux))

NameError: name 'ModelF3' is not defined